<a href="https://colab.research.google.com/github/athar223/Histogram-and-Intensity-Transformations-h1-ed4.ipynb/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install gradio groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import json
import re
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import gradio as gr
from groq import Groq

In [4]:
# Load JSON Dataset
def load_json_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    intents = data["intents"]
    questions, answers = [], []
    for intent in intents:
        for pattern in intent["patterns"]:
            questions.append(pattern)
            answers.append(" ".join(intent["responses"]))
    return questions, answers

In [5]:
# Preprocessing function
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[.!?]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [6]:
# Load dataset
file_path = "/content/drive/MyDrive/data,,/intents.json"  # Update the path
questions, answers = load_json_dataset(file_path)

In [7]:
# Preprocess questions
questions = [preprocess(q) for q in questions]

# Load first model (Existing Medical Chatbot)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(questions))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Create embeddings for all questions
with torch.no_grad():
    question_embeddings = model(**tokenizer(questions, padding=True, truncation=True, return_tensors="pt")).logits

In [9]:
# First Model: Get response from the initial chatbot
def get_response(user_input):
    user_input = preprocess(user_input)
    with torch.no_grad():
        user_embedding = model(**tokenizer([user_input], padding=True, truncation=True, return_tensors="pt")).logits
    similarities = torch.cosine_similarity(user_embedding, question_embeddings, dim=1)
    best_match_index = similarities.argmax().item()
    return answers[best_match_index]

In [10]:
# Initialize the Groq client with your API key
client = Groq(
    api_key="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",
)

# Define a function to get a response using Groq API
def get_groq_response(user_input):
    try:
        # Create a chat completion request
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a medical expert. Only answer medical-related questions. If the question is unrelated to the medical field, respond with 'This is outside the scope of medical knowledge.'",
                },
                {
                    "role": "user",
                    "content": user_input,
                },
            ],
            model="llama3-8b-8192",  # Ensure this model is supported by Groq
        )
        # Extract the first choice's message content
        assistant_message = chat_completion.choices[0].message.content
        return assistant_message
    except AttributeError as e:
        return "Error in accessing the chat completion response: " + str(e)
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [11]:
# Combined Chatbot Functionality
def combined_chatbot(user_input):
    # Step 1: Get response from the first model
    primary_response = get_response(user_input)
    # Step 2: Validate or refine response with the second model
    validated_response = get_groq_response(user_input)
    return f" Assistant: {validated_response}"


In [12]:
# Gradio Interface
interface = gr.Interface(
    fn=combined_chatbot,
    inputs=gr.Textbox(label="Your Medical Query", placeholder="Type your question here..."),
    outputs=gr.Textbox(label="Chatbot Response"),
    title="AI-Powered Medical Chatbot",
    description="Ask questions about diseases, symptoms, or prevention. The system will provide medical-specific responses or indicate if the question is out of scope.",
)

# Launch the Gradio Interface
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://718b6567ef9152416f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
